In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
os.environ["TORCH_DISABLE_DYNAMO"] = "1"


In [4]:
import os
import numpy as np
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm


In [12]:
class SatelliteDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.image_ids = os.listdir(image_dir)
        self.transform = transform

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        img_name = self.image_ids[idx]
        img_path = os.path.join(self.image_dir, img_name)

        try:
            img = Image.open(img_path).convert("RGB")
        except Exception:
            # return a black image instead of crashing
            img = Image.new("RGB", (224, 224), (0, 0, 0))

        if self.transform:
            img = self.transform(img)

        return img, img_name


In [13]:
IMAGE_SIZE = 224

image_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet50(weights=None)
model.fc = nn.Identity()
model = model.to(device)
model.eval()



ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [15]:
TRAIN_IMAGE_DIR = "/kaggle/input/train-naip-images"

BATCH_SIZE = 32

train_dataset = SatelliteDataset(
    image_dir=TRAIN_IMAGE_DIR,
    transform=image_transform
)

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4
)

print("Total train images:", len(train_dataset))


Total train images: 16110


In [16]:
def extract_embeddings(dataloader, model):
    embeddings = []
    image_names = []

    with torch.no_grad():
        for images, names in tqdm(dataloader):
            images = images.to(device)
            feats = model(images)
            embeddings.append(feats.cpu().numpy())
            image_names.extend(names)

    return np.vstack(embeddings), image_names


train_embeddings, train_image_ids = extract_embeddings(train_loader, model)


100%|██████████| 504/504 [36:10<00:00,  4.31s/it]


In [20]:
np.save("/kaggle/working/train_image_embeddings.npy", train_embeddings)
np.save("/kaggle/working/train_image_ids.npy", np.array(train_image_ids))



In [21]:
print(train_embeddings.shape)


(16110, 2048)


In [22]:
TEST_IMAGE_DIR = "/kaggle/input/test-naip-images"


In [23]:
BATCH_SIZE = 32

test_dataset = SatelliteDataset(
    image_dir=TEST_IMAGE_DIR,
    transform=image_transform
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,      # VERY IMPORTANT
    num_workers=4
)

print("Total test images:", len(test_dataset))


Total test images: 5326


In [24]:
test_embeddings, test_image_ids = extract_embeddings(test_loader, model)


100%|██████████| 167/167 [10:38<00:00,  3.82s/it]


In [25]:
import numpy as np

np.save("/kaggle/working/test_image_embeddings.npy", test_embeddings)
np.save("/kaggle/working/test_image_ids.npy", np.array(test_image_ids))
